<a href="https://colab.research.google.com/github/mjn6862/Freiburg_dataset/blob/master/Warping_for_PoseNet_Freiburg%2C_warping_only_(no_data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Statements**

In [ ]:
!pip install tensorflow_graphics

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_graphics.geometry.transformation as tfg_transformation
from tensorflow import keras
tf.keras.backend.set_floatx('float64')
from datetime import datetime
import csv
from matplotlib import pyplot as plt

**Define the input layers**

**Create HTM from 6x1**

Equations taken from DVO paper

**Define the model**

In [2]:
input_A = tf.keras.layers.Input(shape=(480, 640, 3), name="first_image")
depth_A = tf.keras.layers.Input(shape=(480, 640, 1), name="first_depth")
input_B = tf.keras.layers.Input(shape=(480, 640, 3), name="second_image")
depth_B = tf.keras.layers.Input(shape=(480, 640, 1), name="second_depth")

In [3]:
out_dim = 6
concat = tf.keras.layers.concatenate([input_A, input_B])
conv1 = tf.keras.layers.Conv2D(16,(7, 7), padding='same', strides=(2, 2), activation='relu')(concat)
conv2 = tf.keras.layers.Conv2D(32, (5, 5), padding='same', strides=(2, 2), activation='relu')(conv1)
conv3 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', strides=(2, 2), activation='relu')(conv2)
conv4 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', strides=(2, 2), activation='relu')(conv3)
conv5 = tf.keras.layers.Conv2D(256,(3, 3), padding='same', strides=(2, 2), activation='relu')(conv4)
conv6 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', strides=(2, 2), activation='relu')(conv5)
conv7 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', strides=(2, 2), activation='relu')(conv6)
readout = tf.keras.layers.Conv2D(out_dim, (1, 1))(conv7)
readout = 0.1*tf.keras.layers.AveragePooling2D((4,5))(readout)
readout = tf.keras.layers.Flatten()(readout)       # first three elements correspond to the rotational velocity and the other three to the traslational velocity
#htm = batch_mat_exp(readout)
#image_A = tf.keras.backend.reshape(input_A,(1,480,640,3))
#out_img = warp_image(image_A, depth_A, htm)
#model = tf.keras.models.Model(inputs=[input_A, depth_A, input_B, depth_B], outputs=[htm])


In [4]:
# Define hx, wx, h1 as the homogeneous pixel coordinates
# This appears to match the original paper's output format
height_coord = range(480)
width_coord = range(640)
hx, wx = pixel_coord = np.meshgrid(height_coord, width_coord, indexing='ij')
hx = hx.reshape((480, 640, 1))
wx = wx.reshape((480, 640, 1))
h1 = np.ones(hx.shape)

grid = np.concatenate((hx, wx, h1), axis=-1)

# Define the camera intrinsic matrix
#K = np.array([[525.,0.,319.5], [0., 525., 239.5], [0.,0.,1.]])  # This is how the rostopic defines it
K = np.array([[525.,0.,239.5], [0., 525., 319.5], [0.,0.,1.]])

# Define the camera coordinates of each piel up to a depth scale factor
unscaled_cam_coord = np.empty(grid.shape)
for i in range(480):
      for j in range(640):
        unscaled_cam_coord[i,j,:]= np.linalg.inv(K).dot(grid[i,j,:])

**Starting here: layers to go from 6-dim vector to 4x4 HTM within model**


In [30]:
input_xi = tf.keras.layers.Input(shape=(6), name="xi")
input_pose = tf.keras.layers.Flatten()(input_xi) # just a layer that will make things easier to connect later; replace input_xi with readout to connect to bigger model

In [31]:
# create 3x3 matrix omega_x from xi=[omega nu]
omega_x_weights = np.zeros([6,9])
omega_x_weights[0,5]=-1
omega_x_weights[0,7]=1
omega_x_weights[1,2]=1
omega_x_weights[1,6]=-1
omega_x_weights[2,1]=-1
omega_x_weights[2,3]=1

omega_x = tf.keras.layers.Dense(9,use_bias=False,weights=[omega_x_weights],trainable=False)(input_pose)
omega_x = tf.keras.layers.Reshape((3,3))(omega_x)

In [32]:
# create 3-dimensional vector nu from xi=[omega nu]
nu_weights = np.zeros([6,3])
nu_weights[3,0]=1
nu_weights[4,1]=1
nu_weights[5,2]=1

nu = tf.keras.layers.Dense(3,use_bias=False,weights=[nu_weights],trainable=False)(input_pose)
#nu = tf.keras.layers.Reshape((3,1))(nu)

In [33]:
# get ||omega|| from xi
def om_norm_fun(xi):
  return tf.norm(xi[0,0:3])
om_norm = tf.keras.layers.Lambda(om_norm_fun)(input_pose)

In [34]:
# get omega_x**2
omega_x_squared = tf.keras.layers.Dot(axes=(1,2))([omega_x,omega_x])

In [121]:
# get terms for exp(omega_x) and V
# exp(omega_x) = I + term1 + term2 in (2.21)
def term1_fun(x):
  return tf.sin(x[1])/tf.math.maximum(x[1],1e-16)*x[0] #tf.math.maximum to avoid dividing by 0
term1 = tf.keras.layers.Lambda(term1_fun)((omega_x,om_norm))

def term2_fun(x):
  return (1-tf.cos(x[1]))/tf.math.maximum(x[1],1e-16)**2*x[0] #tf.math.maximum to avoid dividing by 0
term2 = tf.keras.layers.Lambda(term2_fun)((omega_x_squared,om_norm))

# V = I + term3 + term4 in (2.22)
def term3_fun(x):
  return (1-tf.cos(x[1]))/tf.math.maximum(x[1],1e-16)**2*x[0] #tf.math.maximum to avoid dividing by 0
term3 = tf.keras.layers.Lambda(term3_fun)((omega_x,om_norm))

def term4_fun(x):
  return (x[1]-tf.sin(x[1]))/tf.math.maximum(x[1],1e-16)**3*x[0] #tf.math.maximum to avoid dividing by 0
term4 = tf.keras.layers.Lambda(term4_fun)((omega_x_squared,om_norm))

In [122]:
# get exp(omega_x)
# exp_om_x = I + term1 + term2
exp_om_x = tf.keras.layers.add([term1,term2])
# still need to add I.  There might be a more elegant way, but I'm going to do 
# this by using a Dense layer and specifying the weights
exp_om_x = tf.keras.layers.Flatten()(exp_om_x)
# use identity for weights to keep the same, then use np.array([1,0,0,0,1,0,0,0,1]) to add identity matrix to exp_om_x
exp_om_x = tf.keras.layers.Dense(9,weights=[np.eye(9),np.array([1,0,0,0,1,0,0,0,1])],trainable=False)(exp_om_x)
exp_om_x = tf.keras.layers.Reshape((3,3))(exp_om_x)

In [123]:
# Get V and V*nu
# V = I + term3 + term4
V = tf.keras.layers.add([term3,term4])
# still need to add I.  There might be a more elegant way, but I'm going to do 
# this by using a Dense layer and specifying the weights
V = tf.keras.layers.Flatten()(V)
# use identity for weights to keep the same, then use np.array([1,0,0,0,1,0,0,0,1]) to add identity matrix to exp_om_x
V = tf.keras.layers.Dense(9,weights=[np.eye(9),np.array([1,0,0,0,1,0,0,0,1])],trainable=False)(V) 
V = tf.keras.layers.Reshape((3,3))(V)
# get V*nu
V_nu = tf.keras.layers.Dot(axes=(2,1))([V,nu])
V_nu = tf.keras.layers.Reshape((3,1))(V_nu)

In [124]:
# get exp_xi by concatenating exp_om_x and V_nu, then concatenate a bottom row of [0,0,0,1]
exp_xi = tf.keras.layers.concatenate([exp_om_x,V_nu])
# this is a silly way to get a bottom row of [0,0,0,1], but it works
bottom_row = tf.keras.layers.Dense(4,weights=[np.zeros([6,4]),np.array([0,0,0,1])],trainable=False)(input_pose)
bottom_row = tf.keras.layers.Reshape((1,4))(bottom_row)

exp_xi = tf.keras.layers.concatenate([exp_xi,bottom_row],axis=1)

In [132]:
#w = tf.constant([np.random.rand(480,640,3),np.random.rand(480,640,3)])
#x = tf.constant([np.random.rand(480,640,1),np.random.rand(480,640,1)])
#y = tf.constant([np.random.rand(480,640,3),np.random.rand(480,640,3)])
#z = tf.constant([np.random.rand(480,640,1),np.random.rand(480,640,1)])
#model = tf.keras.models.Model(inputs=[input_A,depth_A,input_B,depth_B],outputs=[exp_xi])
#model.predict([w,x,y,z])
#xi = tf.constant([[0.3,0.2,0.6,0.8,0.5,0.7]])
xi = tf.constant([[0.1,0,0,1,2,3]])
#xi = tf.constant(np.random.rand(3,6))
model = tf.keras.models.Model(inputs=[input_xi],outputs=[exp_xi])
model.predict(xi)

array([[[ 1.        ,  0.        ,  0.        ,  1.        ],
        [ 0.        ,  0.99500417, -0.09983342,  1.84679329],
        [ 0.        ,  0.09983342,  0.99500417,  3.0949192 ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]])

**Here starts the attempts to warp from input_A, depth_A, and exp_xi**

In [40]:
# Define hx, wx, h1 as the homogeneous pixel coordinates
# This appears to match the original paper's output format
height_coord = range(480)
width_coord = range(640)
hx, wx = pixel_coord = np.meshgrid(height_coord, width_coord, indexing='ij')
hx = hx.reshape((480, 640, 1))
wx = wx.reshape((480, 640, 1))
h1 = np.ones(hx.shape)

grid = np.concatenate((hx, wx, h1), axis=-1)

# Define the camera intrinsic matrix
#K = np.array([[525.,0.,319.5], [0., 525., 239.5], [0.,0.,1.]])  # This is how the rostopic defines it
K = np.array([[525.,0.,239.5], [0., 525., 319.5], [0.,0.,1.]])

# Define the camera coordinates of each piel up to a depth scale factor
unscaled_cam_coord = np.empty(grid.shape)
for i in range(480):
      for j in range(640):
        unscaled_cam_coord[i,j,:]= np.linalg.inv(K).dot(grid[i,j,:])

In [41]:
def warp_image(tensor):
  image1 = tensor[0]
  depth1 = tensor[1]
  delta_pose = tensor[2]
  # This function takes an image and depth map and predicts the view after a transformation by delta_pose
  # What would image1 taken at the location after delta_pose look like? Should match image2
  new_image = tf.keras.backend.ones_like(image1)
  camera_coord = tf.reshape(depth1[-1,:,:],(307200,1))*unscaled_cam_coord.reshape(307200, 3)
  homog_camera_coord = tf.keras.backend.concatenate((camera_coord, tf.keras.backend.ones((307200,1),dtype=tf.float64)) )
  pixel_list = tf.keras.backend.zeros((307200, 2))
  new_homog_camera_coord = tf.keras.backend.dot(homog_camera_coord, tf.keras.backend.transpose(tf.reshape(delta_pose,(4,4))))
  new_homog_camera_coord = tf.math.divide(new_homog_camera_coord, tf.reshape(new_homog_camera_coord[:,2], (307200,1)) ) 
  #print('new_homog_camera_coord=',new_homog_camera_coord)
  pixel_list = tf.keras.backend.dot(tf.reshape(new_homog_camera_coord[:,0:3],(307200,3)), tf.keras.backend.transpose(K))
  #print('pixel_list=',pixel_list)
  print(tf.reshape(image1[-1, :, :, :], (1, 480, 640,3)))
  print(tf.reshape(pixel_list[:,0:2], (-1,307200,2)))
  interp = tfa.image.interpolate_bilinear(grid=tf.reshape(image1[-1, :, :, :], (1, 480, 640,3)), query_points=tf.reshape(pixel_list[:,0:2], (-1,307200,2)))
  #print('interp=',interp)
  new_image = tf.reshape(interp, (-1,480, 640, 3))
  #print('new_image=',new_image)
  #print('camer_coord=',camera_coord)
  #print('homog_camera_coord=',homog_camera_coord)
  #print('pixel_list=',pixel_list)
  #print('new_homog_camera_coord=',new_homog_camera_coord)
  #print('interp=',interp)
  #print('new_image=',new_image)
  return new_image#, camera_coord, homog_camera_coord, new_homog_camera_coord, pixel_list, interp

In [42]:
out = tf.keras.layers.Lambda(warp_image, name='warp_layer')([input_A, depth_A, exp_xi])

Tensor("warp_layer/Reshape_4:0", shape=(1, 480, 640, 3), dtype=float64)
Tensor("warp_layer/Reshape_5:0", shape=(1, 307200, 2), dtype=float64)


ValueError: ignored

In [43]:
camera_coord = tf.keras.layers.Reshape((307200,1),name='camera_coord1')(depth_A)
def camera_coord_fun(depth):
  return depth*unscaled_cam_coord.reshape(307200,3)
  
camera_coord = tf.keras.layers.Lambda(camera_coord_fun,name='camera_coord2')(camera_coord)

In [44]:
# Here's an annoying way to get an output of all ones.  I used a layer with 1 neuron as an intermediary to avoid too many weights
ones_layer = tf.keras.layers.Flatten(name='ones1')(depth_A)
ones_layer = tf.keras.layers.Dense(1,use_bias=False,weights=[np.zeros([307200,1])],trainable=False,name='ones2')(ones_layer)
ones_layer = tf.keras.layers.Dense(307200,weights=[np.zeros([1,307200]),np.ones(307200)],trainable=False,name='ones3')(ones_layer)
ones_layer = tf.keras.layers.Reshape((307200,1),name='ones4')(ones_layer)

In [45]:
homog_camera_coord = tf.keras.layers.concatenate([camera_coord, ones_layer],name='homog_camera_coord')

In [46]:
 # new_homog_camera_coord = tf.keras.backend.dot(homog_camera_coord, tf.keras.backend.transpose(tf.reshape(delta_pose,(4,4))))
 # new_homog_camera_coord = tf.math.divide(new_homog_camera_coord, tf.reshape(new_homog_camera_coord[:,2], (307200,1)) ) 
 new_homog_camera_coord = tf.keras.layers.Dot(axes=(2,2),name='new_homog_camera_coord1')([homog_camera_coord,exp_xi]) # not sure if axes value is correct, but the output has all ones on the right side, so it's probably right

 new_coord_depth = tf.keras.layers.Lambda(lambda coord: coord[:,:,2],name='new_coord_depth1')(new_homog_camera_coord)
 new_coord_depth = tf.keras.layers.Reshape((307200,1),name='new_coord_depth2')(new_coord_depth)
 def new_homog_camera_coord_fun(x):
   return tf.math.divide(x[0],x[1])
new_homog_camera_coord = tf.keras.layers.Lambda(new_homog_camera_coord_fun,name='new_homog_camera_coord2')((new_homog_camera_coord,new_coord_depth))
new_homog_camera_coord = tf.keras.layers.Lambda(lambda coord: coord[:,:,0:3],name='new_homog_camera_coord3')(new_homog_camera_coord)

In [47]:
#  pixel_list = tf.keras.backend.dot(tf.reshape(new_homog_camera_coord[:,0:3],(307200,3)), tf.keras.backend.transpose(K))
# I guess I need to create a layer corresponding to K first
# I used a layer with 1 neuron as an intermediary to avoid too many weights
K_layer = tf.keras.layers.Flatten(name='K1')(depth_A)
K_layer = tf.keras.layers.Dense(1,use_bias=False,weights=[np.zeros([307200,1])],trainable=False,name='K2')(K_layer)
K_layer = tf.keras.layers.Dense(9,weights=[np.zeros([1,9]),K.reshape(9)],trainable=False,name='K3')(K_layer)
K_layer = tf.keras.layers.Reshape((3,3),name='K4')(K_layer)

pixel_list = tf.keras.layers.Dot(axes=(2,2),name='pixel_list1')([new_homog_camera_coord,K_layer]) # not sure if axes value is correct
pixel_list = tf.keras.layers.Lambda(lambda pix: pix[:,:,0:2],name='pixel_list2')(pixel_list)
#pixel_list = tf.keras.layers.Reshape((1,307200,2))(pixel_list)

In [60]:
# interp = tfa.image.interpolate_bilinear(grid=tf.reshape(image1[-1, :, :, :], (1, 480, 640,3)), query_points=tf.reshape(pixel_list[:,0:2], (-1,307200,2)))
#grid_A = tf.keras.layers.Reshape((1,480,640,3))(input_A)
def interp_fun(x):
  return tfa.image.interpolate_bilinear(grid=tf.reshape(x[0],(-1,480,640,3)),query_points=tf.reshape(x[1],(-1,307200,2)))
interp = tf.keras.layers.Lambda(interp_fun,name='interp')((input_A,pixel_list))
new_image = tf.keras.layers.Reshape((480,640,3),name='new_image')(interp)

In [61]:
interp.shape

TensorShape([None, 307200, None])

In [62]:
new_image.shape

TensorShape([None, 480, 640, 3])

In [63]:
pixel_list.shape

TensorShape([None, 307200, 2])

In [ ]:
w = tf.constant([np.random.rand(480,640,3),np.random.rand(480,640,3)])
model = tf.keras.models.Model(inputs=[input_A],outputs=[input_A])
model.predict([w]).shape

(2, 480, 640, 3)

In [64]:
w = tf.constant([np.random.rand(480,640,3),np.random.rand(480,640,3)])
x = tf.constant([np.random.rand(480,640,1),np.random.rand(480,640,1)])
y = tf.constant([np.random.rand(6),np.random.rand(6)])
model = tf.keras.models.Model(inputs=[input_A,depth_A,input_xi],outputs=[interp])
model.predict([w,x,y]).shape

(2, 307200, 3)

In [78]:
w = tf.constant([np.random.rand(480,640,3)])
x = tf.constant([np.random.rand(480,640,1)])
y = tf.constant([np.random.rand(6)])
model = tf.keras.models.Model(inputs=[input_A,depth_A,input_xi],outputs=[interp])
pred = model.predict([w,x,y])

In [101]:
w = tf.constant([np.zeros([480,640,3])])
x = tf.constant([np.ones([480,640,1])])
y = tf.constant([np.zeros(6)])
model = tf.keras.models.Model(inputs=[input_A,depth_A,input_xi],outputs=[new_image])
pred = model.predict([w,x,y])

In [102]:
np.sum(pred)

0.0